# Data wrangling

Since the data tables on the [Footballguys](https://www.footballguys.com/) website are fully rendered in HTML, we might be able to scrape the data without too much trouble. This gives us good control over exactly what data we download and an easy mechanism by which to update it throughout the season. Let's give it a try using [urllib](https://docs.python.org/3/howto/urllib2.html) and [BeautifulSoup](https://beautiful-soup-4.readthedocs.io/en/latest/).

In [36]:
import time
import urllib.request
from itertools import product
from random import randrange

import pandas as pd
from bs4 import BeautifulSoup

## 1. Download and parse HTML data

The available data spans 1996 to 2024 and each year has 18 weeks of data. We also will want to download the data for multiple positions. But, let's start with just one. We also need to pick a scoring scheme, let's go with PPR. We can easily change this later. We will use a loop to construct and download the URL for each year and week and parse and collect the data as we get it.

**Note**: Downloading all of the data for one position takes just over 45 minutes.

### 1.1. Download function

In [38]:
def download_url(url: str) -> bytes:
    '''Takes string url, downloads URL and returns HTML bytes object'''

    headers={
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
        "Accept-Language": "en-US,en;q=0.9",
        "Connection": "keep-alive",
        "Host": "httpbin.io",
        "Sec-Ch-Ua": '"Google Chrome";v="131", "Chromium";v="131", "Not_A Brand";v="24"',
        "Sec-Ch-Ua-Mobile": "?0",
        "Sec-Ch-Ua-Platform": '"Linux"',
        "Sec-Fetch-Dest": "document",
        "Sec-Fetch-Mode": "navigate",
        "Sec-Fetch-Site": "cross-site",
        "Sec-Fetch-User": "?1",
        "Upgrade-Insecure-Requests": "1",
        "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36"
    }

    # Create the request
    request_params = urllib.request.Request(
        url=url,
        headers=headers
    )   

    # Get the html
    with urllib.request.urlopen(request_params) as response:
        html=response.read()

    return html

### 1.2. HTML parsing function

In [39]:
def parse_html_table(html: bytes, year: int, week: int, position: str, profile: str) -> pd.DataFrame:
    '''Takes a html bytes object from URL, parses data table, adds
    year, week, position and scoring profile and returns as pandas dataframe'''

    # Extract the table rows
    soup=BeautifulSoup(html, 'html.parser')
    table=soup.find('table',{'class':'datasmall table'})
    table_rows=table.find_all('tr')

    # Get the column names from the first row
    columns=table_rows[0].find_all('th')
    column_names=[column.getText() for column in columns]
    column_names.extend(['Position', 'Year', 'Week', 'Scoring profile'])

    # Get the values for each row
    data=[]

    for row in table_rows[1:]:
        columns=row.find_all('td')
        values=[column.getText() for column in columns]
        values.extend([position, year, week, profile])
        data.append(values)

    # Convert to pandas dataframe and return
    return pd.DataFrame(columns=column_names, data=data)

### 1.3. Main download loop

In [40]:
# Main script to download data
download_data = True  # Or False, depending on what you want to do

if download_data is True:
    positions = ['qb', 'rb', 'wr', 'te']
    profile = 'p'
    years = list(range(2020, 2024))
    weeks = list(range(1, 19))

    results = []

    for position, year, week in product(positions, years, weeks):
        print(f'Downloading {position.upper()}, {year}, week {week}')
        url = f'https://www.footballguys.com/playerhistoricalstats?pos={position}&yr={year}&startwk={week}&stopwk={week}&profile={profile}'
        
        # Get the HTML
        html = download_url(url)
        
        # Parse the HTML
        result = parse_html_table(html, position, year, week, profile)
        
        # Collect the result
        results.append(result)

        # Wait before downloading the next page
        time.sleep(randrange(1, 5))

    # Combine the week-by-week dataframes
    data_df = pd.concat(results)

elif download_data is False:
    data_df = pd.read_parquet('../data/raw_qb_data.parquet')
    print('Loaded data from file')

# View the resulting DataFrame
print(data_df.head())

  Rank                Name   Age   Exp  G Cmp Att   Cm%  PYd  Y/Att  ...  \
0    1      Josh Allen BUF  24.0   3.0  1  33  46  71.7  312   6.78  ...   
1    2  Russell Wilson SEA  32.0   9.0  1  31  35  88.6  322   9.20  ...   
2    3    Aaron Rodgers GB  37.0  16.0  1  32  44  72.7  364   8.27  ...   
3    4   Lamar Jackson BAL  23.0   3.0  1  20  25  80.0  275  11.00  ...   
4    5    Kyler Murray ARI  23.0   2.0  1  26  40  65.0  230   5.75  ...   

  FantPt Position Year  Week Scoring profile Y/Rsh  Rec  RecYd RecTD  Y/Rec  
0   32.2        1   qb  2020               p   NaN  NaN    NaN   NaN    NaN  
1   31.8        1   qb  2020               p   NaN  NaN    NaN   NaN    NaN  
2   30.8        1   qb  2020               p   NaN  NaN    NaN   NaN    NaN  
3   27.5        1   qb  2020               p   NaN  NaN    NaN   NaN    NaN  
4   26.3        1   qb  2020               p   NaN  NaN    NaN   NaN    NaN  

[5 rows x 26 columns]


## 2. Fix the player name/team column

In [35]:
test_df = data_df.copy()
test_df[['Player', 'Team']] = test_df['Name'].str.extract(r'^(.*?\b(?:I{1,3}|IV)?)(?:\s+)([A-Z]{2,3})$')
test_df.drop(columns=['Name'], inplace=True)
test_df.rename(columns={'Position': 'Week', 'Year': 'Position', 'Week': 'Year'}, inplace=True)
test_df.head()

,Rank,Age,Exp,G,Cmp,Att,Cm%,PYd,Y/Att,PTD,...,RshYd,RshTD,FP/G,FantPt,Week,Position,Year,Scoring profile,Player,Team
0,1,28.0,7.0,1,18,23,78.3,232,10.09,2,...,39,2,33.2,33.2,1,qb,2024,p,Josh Allen,BUF
1,2,29.0,7.0,1,24,30,80.0,289,9.63,4,...,21,0,29.7,29.7,1,qb,2024,p,Baker Mayfield,TB
2,3,24.0,1.0,1,17,24,70.8,184,7.67,0,...,88,2,28.2,28.2,1,qb,2024,p,Jayden Daniels,WAS
3,4,27.0,7.0,1,26,41,63.4,273,6.66,1,...,122,0,27.1,27.1,1,qb,2024,p,Lamar Jackson,BAL
4,5,22.0,2.0,1,9,19,47.4,212,11.16,2,...,56,1,26.1,26.1,1,qb,2024,p,Anthony Richardson,IND


In [15]:
#data_df['Team']=data_df['Name'].apply(lambda x: x.split()[-1])
#data_df['Name']=data_df['Name'].apply(lambda x: ' '.join(x.split()[:-1]))
#data_df.head(10)


IndexError: list index out of range

## 3. Save the data

In [41]:
# First, clean up the index and take a look at what we have:
data_df.reset_index(inplace=True, drop=True)
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26324 entries, 0 to 26323
Data columns (total 26 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Rank             26324 non-null  object
 1   Name             26324 non-null  object
 2   Age              26324 non-null  object
 3   Exp              26324 non-null  object
 4   G                26324 non-null  object
 5   Cmp              2664 non-null   object
 6   Att              2664 non-null   object
 7   Cm%              2664 non-null   object
 8   PYd              2664 non-null   object
 9   Y/Att            2664 non-null   object
 10  PTD              2664 non-null   object
 11  Int              2664 non-null   object
 12  Rsh              19841 non-null  object
 13  RshYd            19841 non-null  object
 14  RshTD            19841 non-null  object
 15  FP/G             26324 non-null  object
 16  FantPt           26324 non-null  object
 17  Position         26324 non-null

In [42]:
# Save as parquet
data_df.to_parquet('../data/raw_qb_data.parquet')

ImportError: Unable to find a usable engine; tried using: 'pyarrow', 'fastparquet'.
A suitable version of pyarrow or fastparquet is required for parquet support.
Trying to import the above resulted in these errors:
 - Missing optional dependency 'pyarrow'. pyarrow is required for parquet support. Use pip or conda to install pyarrow.
 - Missing optional dependency 'fastparquet'. fastparquet is required for parquet support. Use pip or conda to install fastparquet.